In [37]:
import sys
sys.path.insert(0, '/Users/ahmedbegga/Desktop/TFG-Ahmed/SetXAI/src')
import torch
import torch.nn as nn
import torch.nn.functional as F
from model import MaxEncoder
import numpy as np
import h5py
from torch.utils.data import DataLoader, Dataset, TensorDataset
from pointcloud_utils import pcshow , visualize_rotate, read_off
from Modelnet10toSet import *
import plotly.graph_objects as go
import matplotlib as plt

In [18]:
path = Path("../DeepSets Clasification PointCloud/10/raw")

folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
classes

{'bathtub': 0,
 'bed': 1,
 'chair': 2,
 'desk': 3,
 'dresser': 4,
 'monitor': 5,
 'night_stand': 6,
 'sofa': 7,
 'table': 8,
 'toilet': 9}

In [19]:
with open(path/"sofa/train/sofa_0002.off", 'r') as f:
    verts, faces = read_off(f)
    
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T
len(x)

6433

In [20]:
visualize_rotate([go.Scatter3d(x=x, y=y, z=z, mode='markers')]).show()

In [21]:
pointcloud = PointSampler(2048)((verts, faces))
pcshow(*pointcloud.T,c ='navy')

In [22]:
norm_pointcloud = Normalize()(pointcloud)
pcshow(*norm_pointcloud.T)

In [24]:
rot_pointcloud = RandRotation_z()(norm_pointcloud)
noisy_rot_pointcloud = RandomNoise()(rot_pointcloud)
pcshow(*noisy_rot_pointcloud.T)

In [25]:
train_transforms = transforms.Compose([
                    PointSampler(5000),
                    Normalize(),
                    RandRotation_z(),
                    RandomNoise(),
                    ToTensor()
                    ])

In [26]:
train_ds = PointCloudData(path, transform=train_transforms)
valid_ds = PointCloudData(path, valid=True, folder='test', transform=train_transforms)

In [27]:
train_ds[0]

{'pointcloud': tensor([[ 0.0480,  0.5056,  0.0627],
         [ 0.0331, -0.1898, -0.1783],
         [-0.6258,  0.3056,  0.0914],
         ...,
         [-0.0340,  0.5854, -0.0376],
         [-0.4957,  0.1361, -0.0719],
         [ 0.2727, -0.2854, -0.2260]], dtype=torch.float64),
 'category': 0}

In [28]:
inv_classes = {i: cat for cat, i in train_ds.classes.items()};
inv_classes

{0: 'bathtub',
 1: 'bed',
 2: 'chair',
 3: 'desk',
 4: 'dresser',
 5: 'monitor',
 6: 'night_stand',
 7: 'sofa',
 8: 'table',
 9: 'toilet'}

In [29]:
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(valid_ds))
print('Number of classes: ', len(train_ds.classes))
print('Sample pointcloud shape: ', train_ds[0]['pointcloud'].size())
print('Class: ', inv_classes[train_ds[0]['category']])

Train dataset size:  3991
Valid dataset size:  908
Number of classes:  10
Sample pointcloud shape:  torch.Size([5000, 3])
Class:  bathtub


In [30]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=64)

In [31]:
data_iter = iter(train_loader)
data = data_iter.next()
set3D,label3D = data['pointcloud'],data['category']
set3D[20].shape

torch.Size([5000, 3])

In [32]:
label3D[3]

tensor(7)

In [33]:
pcshow(*set3D[3].transpose(0,1),c='red')

In [39]:
# No se puede visualizar lo de arriba en github, asi que usaremos matplot para que se vea eb github
a = set3D[3].transpose(0,1).numpy

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(a[0,:], a[1,:], a[2, :],color = 'r')
ax.set_axis_off()
plt.show()

TypeError: 'module' object is not callable